In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tqdm import tqdm
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [2]:
GPU = True

device = "cuda" if GPU else "cpu"

In [3]:
dataset = datasets.MNIST('./mnist', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST('./mnist', train=False, download=True, transform=transforms.ToTensor())

In [4]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [5]:
class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()

    self.fc1 = nn.Linear(784, 400)
    self.fc21 = nn.Linear(400, 20)
    self.fc22 = nn.Linear(400, 20)
    self.fc3 = nn.Linear(20, 400)
    self.fc4 = nn.Linear(400, 784)

  def encode(self, x):
    h1 = F.relu(self.fc1(x))
    return self.fc21(h1), self.fc22(h1)

  def reparameterize(self, mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    return mu + eps*std

  def decode(self, z):
    h3 = F.relu(self.fc3(z))
    return torch.sigmoid(self.fc4(h3))

  def forward(self, x):
    mu, logvar = self.encode(x.view(-1, 784))
    z = self.reparameterize(mu, logvar)
    return self.decode(z), mu, logvar


In [6]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
def loss_function(recon_x, x, mu, logvar):
  BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction="sum")

  KLD = -0.5*torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

  return BCE + KLD

In [8]:
def train(epoch):
  model.train()
  train_loss = 0
  for (data, _) in tqdm(dataloader):
    data = data.to(device)
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    train_loss += loss.item()
    optimizer.step()

  print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, (train_loss/len(dataloader.dataset)/100)))

In [9]:
def test(epoch):
  model.eval()
  test_loss = 0
  with torch.no_grad():
    for batch_idx, (data, _) in tqdm(enumerate(testloader)):
      data = data.to(device)
      recon_batch, mu, logvar = model(data)
      test_loss += loss_function(recon_batch, data, mu, logvar).item()
      if batch_idx == 0:
        n = min(data.size(0), 8)
        comparison = torch.cat([data[:n], recon_batch.view(4, 1, 28, 28)[:n]])
        save_image(comparison.cpu(), 'result/reconstruction_' + str(epoch) + ".png", nrow=n)
  
  test_loss /= len(testloader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss/100))


In [10]:
def run(mode, epochs):
  if mode == "TRAIN":
    for epoch in range(epochs):
      train(epoch + 1)
  elif mode == "TEST":
    for epoch in range(epochs):
      test(epoch + 1)

In [11]:
if not os.path.exists("result"):
    os.makedirs("result")

In [12]:
EPOCH = 50
MODE = "TRAIN"

run(MODE, EPOCH)

  0%|          | 0/1875 [00:00<?, ?it/s]

====> Epoch: 1 Average loss: 1.3565


  0%|          | 0/1875 [00:00<?, ?it/s]

====> Epoch: 2 Average loss: 1.1298


  0%|          | 0/1875 [00:00<?, ?it/s]

...

====> Epoch: 48 Average loss: 1.0203


  0%|          | 0/1875 [00:00<?, ?it/s]

====> Epoch: 49 Average loss: 1.0194


100%|██████████| 1875/1875 [00:06<00:00, 268.24it/s]

====> Epoch: 50 Average loss: 1.0197


In [13]:
EPOCH = 20
MODE = "TEST"

run(MODE, EPOCH)

2500it [00:03, 807.58it/s]

====> Test set loss: 1.0206



2500it [00:03, 821.35it/s]

====> Test set loss: 1.0206



2500it [00:03, 816.99it/s]

...

====> Test set loss: 1.0203



2500it [00:03, 816.68it/s]

====> Test set loss: 1.0217



2500it [00:03, 821.73it/s]

====> Test set loss: 1.0212


In [14]:
def plot_reconstruct():
    images = [file for file in os.listdir('./result') if os.path.isfile(os.path.join('./result',file))]
    steps = []
    
    for idx in range(len(images)):
        steps.append(imageio.imread("result/reconstruction_" + str(idx+1) + ".png"))
    
    imageio.mimsave('./animation.gif', steps, fps=1)
        
plot_reconstruct()

![SegmentLocal](animation.gif "animation")